In [23]:
using Revise
using ITensors,ITensorMPS
using PastaQ
push!(LOAD_PATH,"../src/")
using RandomMeas
using ProgressBars


In [25]:
N = 10
ξ = siteinds("Qubit", N)
B = 1.
ampo = AutoMPO()
for j in 1:(N - 1)
  # Ising ZZ interactions
  ampo .+= -1, "X", j, "X", j + 1
end
for j in 1:N
  # Transverse field X
  ampo .+= -B, "Z", j
end
H = MPO(ampo,ξ)
H2 = apply(H,H)
# Density-matrix renormalization group
dmrg_iter = 5      # DMRG steps
dmrg_cutoff = 1E-10   # Cutoff
ψ0 = randomMPS(ξ) # Initial state
sweeps = Sweeps(dmrg_iter)
maxdim!(sweeps, 10, 20, 30, 40, 50, 100)
cutoff!(sweeps, dmrg_cutoff)
# Run 
println("Running DMRG to get ground state of transverse field Ising model:")
E, ψ = dmrg(H, ψ0, sweeps)
println("\nGround state energy:  ", E)
println("\n---------------------------------------\n")

Running DMRG to get ground state of transverse field Ising model:
After sweep 1 energy=-12.286585733423948  maxlinkdim=4 maxerr=1.90E-16 time=0.008
After sweep 2 energy=-12.380216814044163  maxlinkdim=13 maxerr=8.29E-11 time=0.012
After sweep 3 energy=-12.381489464429103  maxlinkdim=10 maxerr=9.22E-11 time=0.013
After sweep 4 energy=-12.381489996617532  maxlinkdim=8 maxerr=8.92E-11 time=0.014
After sweep 5 energy=-12.381489996614333  maxlinkdim=8 maxerr=7.01E-11 time=0.031

Ground state energy:  -12.381489996614333

---------------------------------------



In [26]:
η = 10
m = 4
# Build the gate structure
circuit = []
for d in 1:η
    xx_layer = [("Rxx", (j, j + 1), (ϕ=1*d/η/m,)) for j in 1:(N - 1)]
    z_layer = [("Rz", j, (θ=2*B/m,)) for j in 1:N]
    for t in 1:m
        append!(circuit, xx_layer)
        append!(circuit, z_layer)
    end
end
ψ0 = MPS(ξ,["Up" for n in 1:N]);

In [27]:
ψt = runcircuit(ψ0, circuit; cutoff=1E-8)
orthogonalize!(ψt,1)
ψt[1] /= norm(ψt[1])
E_th = real(inner(ψt', H,ψt))
println("final energy ",E_th)
E2_th = real(inner(ψt', H2,ψt))-E_th^2
println("final energy variance ",E2_th)

final energy -11.869405413372245
final energy variance 2.427253542583628


In [28]:
Nu = 1000
NM = 100
ρ = Vector{ITensor}(undef,N);
data = zeros(Int8,(NM,N));

In [35]:
Es = 0.
E2s = 0.
for r in ProgressBar(1:Nu, printing_delay=5)
        u = get_rotations(ξ,1)
        get_RandomMeas_MPS!(data,ψt,u)
        for m in 1:NM
            get_shadow_factorized!(ρ,data[m,:],ξ,u)
            Es += get_expect_shadow(H,ρ,ξ)/Nu/NM
            E2s += get_expect_shadow(H2,ρ,ξ)/Nu/NM
        end
end
E2s -= Es^2
println("estimated energy ",Es)
println("estimated energy variance ",E2s)

0.0%┣                                             ┫ 0/1.0k [00:00<00:00, -0s/it]
11.8%┣█████                                     ┫ 118/1.0k [00:05<00:38, 23it/s]
22.5%┣█████████▌                                ┫ 225/1.0k [00:10<00:35, 22it/s]
34.6%┣██████████████▌                           ┫ 346/1.0k [00:15<00:29, 23it/s]
46.9%┣███████████████████▊                      ┫ 469/1.0k [00:20<00:23, 23it/s]
59.7%┣█████████████████████████                 ┫ 597/1.0k [00:25<00:17, 24it/s]
69.6%┣█████████████████████████████▎            ┫ 696/1.0k [00:30<00:13, 23it/s]
81.6%┣██████████████████████████████████▎       ┫ 816/1.0k [00:35<00:08, 23it/s]
94.6%┣███████████████████████████████████████▊  ┫ 946/1.0k [00:40<00:02, 24it/s]


estimated energy -1.2736076985924307
estimated energy variance 29.670124934968896


100.0%┣████████████████████████████████████████┫ 1.0k/1.0k [00:42<00:00, 24it/s]
100.0%┣████████████████████████████████████████┫ 1.0k/1.0k [00:42<00:00, 24it/s]
